## Problem Statement    
This analysis examines profitability across business segments to identify which segments create or destroy value. Using segment-level cost and pricing data, the analysis isolates the drivers of profitability differences and evaluates whether losses in the weakest-performing segment are attributable to pricing, product mix, or operating cost structure. A counterfactual scenario is used to quantify the financial impact if the unprofitable segment operated under a more efficient cost model.

## Data overview & Preparation  
The dataset contains aggregated product-level sales and cost information across business segments and geographies. Each row represents aggregated product performance rather than individual transactions.  

Initial data quality checks showed low overall missingness across variables. “Discount Band” exhibited the highest proportion of missing values and unclear semantics, prompting further investigation into whether it materially explained differences in profitability.  

Minor data cleaning was performed to standardize categorical identifiers (e.g., country naming inconsistencies) to ensure accurate aggregation. No records were removed, and no values were imputed.

In [25]:
import pandas as pd

#load the dataframe
financial_df = pd.read_excel('SampleFinData.xlsx')

#minimal cleaning: standarize categorical identifiers for accurate grouping
financial_df["Country"] = financial_df["Country"].str.strip().str.title()
financial_df["Country"] = financial_df["Country"].replace({"United States \nOf America" : "United States Of America"})
financial_df["Segment"] = financial_df["Segment"].str.strip().str.title()
financial_df["Segment"] = financial_df["Segment"].replace({"Midmarkets" : "Midmarket"})
financial_df["Product"] = financial_df["Product"].str.strip().str.title()
financial_df["Discount Band"] = financial_df["Discount Band"].str.strip().str.title()

#data quality: missingness summary
missing = financial_df.isna().sum().sort_values(ascending = False)
missing_pct = (financial_df.isna().mean() * 100).sort_values(ascending = False)
missing_summary = pd.DataFrame({
    "missing_count": missing, 
    "missing_%": missing_pct
}).head(15)

missing_summary

,missing_count,missing_%
Discount Band,62,8.720113
Discounts,14,1.969058
Segment,7,0.984529
Country,7,0.984529
Product,7,0.984529
Units Sold,7,0.984529
Manufacturing Price,7,0.984529
Sale Price,7,0.984529
Gross Sales,7,0.984529
Sales,7,0.984529


## Analytics Approach  

Profitability was evaluated using **profit margin (Profit / Sales)** as the primary metric, as it normalizes performance across segments with different revenue scales.  

To determine which dimensions most strongly explained variation in profitability, key categorical variables (Segment, Discount Band, Product, Country) were evaluated individually by comparing the dispersion of profit margins across groups. Standard deviation and range were used as summary measures to assess explanatory power.  

Dimensions exhibiting minimal dispersion were treated as contextual, while dimensions with large dispersion were examined further to identify underlying economic drivers.  

Segment-level analysis was prioritized after dispersion testing indicated it was the most explanatory dimension of profitability.

In [14]:
segment_summary = financial_df.groupby("Segment")[[" Sales", "Profit"]].agg(
    total_sales = (" Sales", "sum"),
    total_profit = ("Profit", "sum"),
    profit_margin = ("Profit", lambda x: x.sum() / financial_df.loc[x.index, " Sales"].sum())
).sort_values("total_sales", ascending = False)

segment_display = segment_summary.rename(columns = {
    "total_sales": "Total Sales",
    "total_profit": "Total Profit",
    "profit_margin": "Profit Margin"
})

segment_display.style.format({
    "Total Sales": "${:,.2f}",
    "Total Profit": "${:,.2f}",
    "Profit Margin": "{:.2%}"
})

,Total Sales,Total Profit,Profit Margin
Segment,,,
Government,"$52,923,525.67","$11,492,838.17",21.72%
Small Business,"$43,169,824.50","$4,260,574.50",9.87%
Enterprise,"$19,611,694.38","$-614,545.62",-3.13%
Midmarket,"$2,391,108.08","$663,178.07",27.74%
Channel Partners,"$1,824,029.64","$1,334,380.14",73.16%


Profit margins vary substantially across segments (ranging from –3% to over 70%), indicating that segment is the most explanatory dimension of profitability.

## Drivers of Profitability  
To distinguish whether pricing or cost structure explains segment-level margin differences, discount rates and cost-of-goods-sold (COGS) were compared across segments.  

Discount rates are relatively consistent across segments, suggesting pricing strategy is not the primary driver of profitability. In contrast, COGS as a percentage of sales varies substantially by segment, indicating that differences in operating cost structure are the dominant factor driving profit margins.

In [19]:
#decompose Sales, Discounts, COGS, Profit Margin
segment_drivers = financial_df.groupby("Segment")[[" Sales", "Discounts", "COGS", "Profit"]].agg(
    discount_rate = ("Discounts", lambda x: x.sum() / financial_df.loc[x.index, " Sales"].sum()),
    cogs_rate = ("COGS", lambda x: x.sum() / financial_df.loc[x.index, " Sales"].sum()),
    profit_margin = ("Profit", lambda x: x.sum() / financial_df.loc[x.index, " Sales"].sum())
)

driver_display = segment_drivers.rename(columns = {
    "discount_rate": "Discount Rate",
    "cogs_rate": "COGS Rate",
    "profit_margin": "Profit Margin"
})
                        
driver_display.style.format({
    "Discount Rate": "{:.2%}",
    "COGS Rate": "{:.2%}",
    "Profit Margin": "{:.2%}"
})

,Discount Rate,COGS Rate,Profit Margin
Segment,,,
Channel Partners,7.38%,26.84%,73.16%
Enterprise,7.43%,103.13%,-3.13%
Government,7.37%,78.28%,21.72%
Midmarket,8.40%,72.26%,27.74%
Small Business,8.16%,90.13%,9.87%


Within the Enterprise segment, losses are consistent across observations rather than driven by a small number of outliers. Further decomposition by product shows that all products are unprofitable within Enterprise, indicating that losses are structural and attributable to the segment’s operating model rather than product mix.

## Counterfactual Analysis  
To quantify the economic impact of Enterprise’s cost structure, a counterfactual scenario was constructed in which Enterprise retains its observed sales volume, product mix, and discounting behavior, but operates at the same cost-of-goods-sold (COGS) rate as the Channel Partners segment.  

This isolates operating efficiency as the sole variable, allowing the impact of cost structure differences to be evaluated independently of pricing or demand.

In [24]:
#Channel Partners cost structure benchmark
cp_cogs_rate = (
    financial_df[financial_df["Segment"] == "Channel Partners"]["COGS"].sum() /
    financial_df[financial_df["Segment"] == "Channel Partners"][" Sales"].sum()
)

#Enterprise actuals
ent = financial_df[financial_df["Segment"] == "Enterprise"]
ent_sales = ent[" Sales"].sum()
ent_discounts = ent["Discounts"].sum()
ent_profit_actual = ent["Profit"].sum()

#Counterfactual Enterprise profit using CP cost structure 
ent_cogs_cf = ent_sales * cp_cogs_rate
ent_profit_cf = ent_sales - ent_discounts - ent_cogs_cf

print(f' Enterprise Profit (Actual): ${ent_profit_actual:,.2f}')
print(f' Enterpirse Profit (Counterfactual): ${ent_profit_cf:,.2f}')

 Enterprise Profit (Actual): $-614,545.62
 Enterpirse Profit (Counterfactual): $12,889,750.43


**Results**

Under current operations, Enterprise generates an aggregate loss of approximately **\$0.6M**.Under the counterfactual scenario, applying Channel Partners’ cost structure to Enterprise revenue converts this loss into approximately **\$12.9M** in profit.

This represents a total swing of roughly **\$13.5M**, attributable entirely to differences in operating cost structure rather than pricing, product mix, or demand.

**What-If Scenario: Enterprise Break-Even Threshold**

To assess a more conservative and potentially feasible improvement, a break-even scenario was evaluated for the Enterprise segment. Holding Enterprise sales volume and discounting constant, the analysis identifies the maximum cost-of-goods-sold (COGS) rate at which Enterprise would generate zero profit.

This analysis indicates that Enterprise must reduce its COGS rate from approximately **103% of sales to ~92.6%** in order to break even. While this improvement is substantially smaller than the gap implied by the Channel Partners benchmark, it still represents a material structural change and highlights the extent to which current Enterprise operations are economically misaligned.


In [35]:
#Company Actuals
total_sales = financial_df[" Sales"].sum()
total_profit_actual = financial_df["Profit"].sum()
profit_margin_actual  = total_profit_actual / total_sales


#Enterprise Actuals
ent_profit_actual = financial_df.loc[
    financial_df["Segment"] == "Enterprise", "Profit"].sum()


#Scenario: Enterprise Breaks-Even (Enterprise profit becomes 0)
total_profit_be = total_profit_actual - ent_profit_actual
profit_margin_be = total_profit_be / total_sales

#Margins are in percentages
print(f'Company Profit (Actual): ${total_profit_actual:,.2f}')
print(f'Company Profit Margin (Actual): {profit_margin_actual:.3%}')
print(f'Company Profit (Break-Even): {total_profit_be:,.2f}')
print(f'Company Profit Margin (Break-Even): {profit_margin_be:.3%}')

Company Profit (Actual): $17,136,425.26
Company Profit Margin (Actual): 14.290%
Company Profit (Break-Even): 17,750,970.88
Company Profit Margin (Break-Even): 14.802%


**Enterprise at Break-Even (Company-Level Impact)**

At current performance levels, the company generates approximately **\$17.1M** in total profit, corresponding to a profit margin of **14.3%**. If the Enterprise segment were to reach a minimum viable state (zero profit), total company profit would increase to approximately **\$17.8M**, with the profit margin improving to **14.8%**.

While this improvement eliminates Enterprise’s value destruction, the relatively modest change at the firm level suggests that Enterprise’s losses are material at the segment level but not the primary driver of overall company profitability.
